In [1]:
# libraries importing
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
import numpy as np

from tensorflow import keras
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, LSTM, GRU, RepeatVector, TimeDistributed
from keras.callbacks import EarlyStopping
import pickle
from tqdm import tqdm 

c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
WINDOW_SIZE=40
def create_sequences(values, time_steps=WINDOW_SIZE):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)


def MODEL():


    model = Sequential()
    model.add(LSTM(144, activation='relu', return_sequences=True, input_shape=(WINDOW_SIZE-1, 19)))
    model.add(LSTM(40, activation='relu'))
    model.add(Dense(19))
    model.compile(optimizer='adam',loss='mse', metrics=["mse"])
    return model



In [4]:
#create model
with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
            data= pickle.load(f)

X_train=data['X_train']
X_val=data['X_val']

model=MODEL()
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=30)

model.summary()

Y=X_train[:,39,:]

Y_val=X_val[:,39,:]


history = model.fit(X_train[:,:39,:],Y, epochs=40, batch_size=32,validation_data=(X_val[:,:39,:],Y_val), verbose=1, callbacks=[callback])
model.save_weights(f"LSTM_MODEL/model_apptot_Win_{WINDOW_SIZE}_LSTM_144_40.h5")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 39, 144)           94464     
                                                                 
 lstm_3 (LSTM)               (None, 40)                29600     
                                                                 
 dense_1 (Dense)             (None, 19)                779       
                                                                 
Total params: 124,843
Trainable params: 124,843
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
1959/1959 [==============================] - ETA: 0s - loss: 0.8347 - mse: 0.8347

In [3]:
import pickle
with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
            DATA = pickle.load(f)
    
model=MODEL()

model.load_weights(filepath=f"LSTM_MODEL/model_apptot_Win_{WINDOW_SIZE}_LSTM_144_40.h5")

e=model.predict(DATA['X_test'][:,:39,:])
score=np.mean(np.power(e-DATA['X_test'][:,39,:],2),axis=(1))

np.save('./score_analisi/LSTM.npy',score,allow_pickle=True)


In [11]:
import csv


header=['th_factor','method','value','F1','precision','recall','TP','TN','FP','FN']

with open('LSTM.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerow(header)
    thresholding_factor=[0.5,1,1.5,2]
    for t in thresholding_factor:
        with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
            DATA = pickle.load(f)
    
        model=MODEL()

        model.load_weights(filepath=f'LSTM_MODEL/model_apptot_Win_{WINDOW_SIZE}_LSTM_144_40_early.h5')


        e=model.predict(DATA['X_test'])
        score=np.mean(np.power(e-DATA['X_test'][:,39,:],2),axis=(1))

        #IQR
        q1, q3 = np.percentile(score, 25), np.percentile(score, 75)
        iqr = q3 - q1
        IQR = q3 + t* iqr

        #MAD
        median = np.median(score)
        mad = 1.4826 * np.median(np.abs(score - median))
        MAD = median + t * mad

        #STD
        mean, std = np.mean(score), np.std(score)
        STD = mean + t * std

        method=[IQR,MAD,STD]
        
        for g in range(len(method)):
            TP=0
            TN=0
            FN=0
            FP=0

            for i in tqdm(range(1,11)):
                if(i!=7):
                
                    TEST=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/test{i}.npy',allow_pickle=True)

                    ANOMALY=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/y_test{i}.npy',allow_pickle=True)

                    with open(f'./OUTPUTS_ROOT/data/interim/spark_0_15s/test_info{i}.pkl', 'rb') as f:
                        TEST_info= pickle.load(f)


                    
                    for x in range(len(TEST_info)):

                        X=create_sequences(TEST[x])

                        Y=model.predict(X)
                        error=np.mean(np.power(X[:,39,:]-Y,2),axis=(1))

                        error=[error[l] if error[l]<2 else 2 for l in range(len(error))]

                        outlier=error>method[g]


                        true_= ANOMALY[x][WINDOW_SIZE-1:]>=1
                        prediction_ = outlier[:].astype(int)==1
                        
                        TP = TP+(true_ & prediction_).sum()   
                        TN = TN+(~true_ & ~prediction_).sum()  
                        FP = FP+(~true_ & prediction_).sum()    
                        FN = FN+(true_ & ~prediction_).sum()  

            PREC=TP / (TP + FP)
            REC = TP/ (TP+FN)
            f1=2 * PREC * REC/(PREC + REC)
            m=['IQR','MAD','STD']
            row=[t,m[g],method[g],f1,PREC,REC,TP,TN,FP,FN]
            writer.writerow(row)

100%|██████████| 10/10 [00:27<00:00,  2.74s/it]
